In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/FDA project/all_matches_2021.csv')

In [ ]:
#calculating and adding all possible runs per ball to new df new column - 'runs'
runs=df['runs_off_bat']+df['extras']
#+df['wides']+df['noballs']+df['byes']+df['legbyes']+df['penalty']
df['runs']=runs

In [ ]:
#calculating and adding runs per innings
total_run_fir_inn=0
total_run_sec_inn=0
df['innings_run']=""

for index, row in df.iterrows():
  #print(index)
  if index>0:
    #print(df['runs'][index])
    if df['match_id'][index]==df['match_id'][index-1]:
      if df['innings'][index]==1:
        total_run_fir_inn+=df['runs'][index]
  
  if df['innings'][index]==1 and df['ball'][index+1]==0.1:
    df['innings_run'][index]=total_run_fir_inn
    total_run_fir_inn=0
  
  if df['innings'][index]==2:
    total_run_sec_inn+=df['runs'][index]

  if index<200663:  
    if df['innings'][index]==2 and df['ball'][index+1]==0.1:
      df['innings_run'][index]=total_run_sec_inn
      total_run_sec_inn=0

  else:
    total_run_fir_inn=0
    total_run_sec_inn=0

In [ ]:
inner_it_1 = 0
df['run_sum']=0
for index, row in df.iterrows():
  if(index<=200662 and index>=0):
    if(df['ball'][index+1]==0.1):
      inner_it_2 = index
      for i in range(inner_it_1,inner_it_2+1):
        df['run_sum'][i]=df['innings_run'][inner_it_2]
      inner_it_1 = inner_it_2+1
  elif(index==200663):
    inner_it_2 = 200663
    for i in range(inner_it_1,inner_it_2+1):
      df['run_sum'][i]=df['innings_run'][inner_it_2]

In [ ]:
#calculation of the runs
runs_tot=[]
runs_a=0
for index, row in df.iterrows():
  if index<=200664 and index>=0:
    if df['ball'][index]==0.1:
      runs_a=0
    
    runs_a+=df['runs'][index]
    runs_tot.append(runs_a)
df['runs_tot']=runs_tot

In [ ]:
df.shape

In [ ]:
#calculating and adding all possible runs per ball to new df new column - 'runs'
runs=df['runs_off_bat']+df['extras']
#+df['wides']+df['noballs']+df['byes']+df['legbyes']+df['penalty']
df['runs']=runs
df.head()

In [ ]:
#df=df[df['innings']==1]

In [ ]:
person_a=df['striker'][0]    
person_b=df['non_striker'][0]
runs_a=0
runs_b=0
list_a=[]
list_b=[]

for index, row in df.iterrows():
  #when there is no change in wicket, initialize the 2 players as a and b
  #exept when it is the start of the over, then initialize the players as a and b
  print(index)
  #initializing the person_a and person_b
  if index<=200663 and index>=0:

    if df['ball'][index]==0.1:
      person_a=df['striker'][index]
      person_b=df['non_striker'][index]
      runs_a=0
      runs_b=0
      #print("visited first if initialize")

    elif not pd.isnull(df.loc[index]['player_dismissed']):
      #change the person who lost the wicket
      #print("visited second if initialize")
      if person_a == df.loc[index]['player_dismissed']:
        runs_a=0
        #update to the new person        
        x=[person_a,person_b]
        new_player=" "
        if df['striker'][index+1] in x:
          new_player=df['non_striker'][index+1]
        else:
          new_player=df['striker'][index+1]        
        person_a=new_player
        
      else:
        runs_b=0
        #update to the new person        
        x=[person_a,person_b]
        new_player=" "
        if df['striker'][index+1] in x:
          new_player=df['non_striker'][index+1]
        else:
          new_player=df['striker'][index+1]        
        person_b=new_player

    #adding the runs to the list
    if df['striker'][index]==person_a:
      # print(runs_a)
      runs_a+=df['runs_off_bat'][index]
      #print(df['runs_off_bat'][index])
      #print("runs scored by a is {} at index {}".format(runs_a,index))
      list_a.append(runs_a)
      list_b.append(runs_b)
    else:
      #print(runs_b)
      runs_b+=df['runs_off_bat'][index]
      #print(df['runs_off_bat'][index])
      #print("runs scored by b is {} at index {}".format(runs_b,index))
      list_a.append(runs_a)
      list_b.append(runs_b)

In [ ]:
df.head()

In [ ]:
required_striker=max(list_a,list_b)
required_non_striker=min(list_a,list_b)

In [ ]:
df['required_striker']=required_striker
df['required_non_striker']=required_non_striker


In [ ]:
df1 = df[['striker','non_striker','Wickets','ball','run_sum','innings','runs_off_bat','player_dismissed','required_striker','required_non_striker','runs_tot','runs']]

In [ ]:
df1.tail(100)

,striker,non_striker,Wickets,ball,run_sum,innings,runs_off_bat,player_dismissed,required_striker,required_non_striker,runs_tot,runs,required_striker_1,required_non_striker_1
200564,PP Shaw,S Dhawan,0,1.3,,2,6,NaN,7,8,16,6,8,7
200565,PP Shaw,S Dhawan,0,1.4,,2,4,NaN,7,12,20,4,12,7
200566,PP Shaw,S Dhawan,0,1.5,,2,4,NaN,7,16,24,4,16,7
200567,PP Shaw,S Dhawan,0,1.6,,2,0,NaN,7,16,24,0,16,7
200568,S Dhawan,PP Shaw,0,2.1,,2,0,NaN,7,16,24,0,16,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200659,SO Hetmyer,S Dhawan,3,17.2,,2,6,NaN,69,6,155,6,69,6
200660,SO Hetmyer,S Dhawan,3,17.3,,2,6,NaN,69,12,161,6,69,12
200661,SO Hetmyer,S Dhawan,3,17.4,,2,4,NaN,69,16,165,4,69,16
200662,SO Hetmyer,S Dhawan,3,17.5,,2,0,NaN,69,16,166,1,69,16


In [ ]:
df1['required_striker_1'] = ""
df1['required_non_striker_1'] = ""
for index, row in df.iterrows():
  df1['required_striker_1'][index] = max(df1['required_striker'][index],df1['required_non_striker'][index])
  df1['required_non_striker_1'][index] = min(df1['required_striker'][index],df1['required_non_striker'][index])


In [ ]:
df1.to_csv("final_PreProcess10.csv")